In [ ]:
# adapted from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
# https://www.youtube.com/watch?v=MqugtGD605k
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense, BatchNormalization
import numpy as np
import wandb
from wandb.keras import WandbCallback
import random

wandb.init()
config = wandb.config

class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.
        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

# Parameters for the model and dataset.
config.hidden_size = 256
config.batch_size = 32

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
maxlen = 25
output_len = 10
# All the numbers, plus sign and space for padding.
chars = '0123456789+$-,/.ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuwxyv '
ctable = CharacterTable(chars)


import pandas as pd
df = pd.read_csv("sale_prices.csv")
df = df.fillna(0)
from sklearn.utils import shuffle
df = shuffle(df)


raw_prices = df["Price on site"].tolist()
parced_prices = []

int_min_price = [int(price) for price in df["Min price"]]
int_max_price = [int(price) for price in df["Max price"]]

# for idx in range(len(int_min_price)):
#     toappend = str(int_min_price[idx])
#     toappend+=' '*(output_len - len(str(int_min_price[idx])))
#     parced_prices.append(toappend)
    
#augmentation (acc: 0.9878)
# for i in range(4):
#     for idx in range(len(int_min_price)):
#         raw_price = str(raw_prices[idx])
#         parsed_price = str(int_min_price[idx])
#         num_of_replacements = random.randint(1,5)
#         for i in range(num_of_replacements):
#             to_replace = str(random.randint(1,9))
#             replace_with = str(random.randint(1,9))
#             parsed_price.replace(to_replace,replace_with)
#             raw_price.replace(to_replace,replace_with)

#         parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
#         parced_prices.append(parsed_price)
#         raw_prices.append(raw_price)

#augmentation v2

split_at = len(int_min_price) - len(int_min_price) // 10

#train data
raw_prices_train = []
parced_prices_train = []

for i in range(1000):  #augmentation x100 times (from ~500 to 500000 ~datapoints)
    for idx in range(split_at):
        raw_price = str(raw_prices[idx])
        parsed_price = str(int_min_price[idx])
        for i in range(len(parsed_price)):
            to_replace = parsed_price[i]
            if i == 0: # to do not make first digit as zero
                replace_with = str(random.randint(1,9))
            else:
                replace_with = str(random.randint(0,9))
            parsed_price = parsed_price.replace(to_replace,replace_with)
            raw_price = raw_price.replace(to_replace,replace_with)

        parsed_price+=' '*(output_len - len(str(int_min_price[idx])))
        parced_prices_train.append(parsed_price)
        raw_prices_train.append(raw_price)
        
#val data
raw_prices_val = []
parced_prices_val = []
for index in range(split_at,len(raw_prices)-1):
    raw_price = str(raw_prices[index])
    parsed_price = str(int_min_price[index])
    for i in range(len(parsed_price)):
        to_replace = parsed_price[i]
        if i == 0: # to do not make first digit as zero
            replace_with = str(random.randint(1,9))
        else:
            replace_with = str(random.randint(0,9))
        parsed_price = parsed_price.replace(to_replace,replace_with)
        raw_price = raw_price.replace(to_replace,replace_with)

    parsed_price+=' '*(output_len - len(str(int_min_price[index])))
    parced_prices_val.append(parsed_price)
    raw_prices_val.append(raw_price)
        
        
print('Total train prices:', len(raw_prices_train))
print('Total validation prices:', len(raw_prices_val))

print('Vectorization...')
x_train = np.zeros((len(raw_prices_train), maxlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(parced_prices_train), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_train):
    x_train[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_train):
    y_train[i] = ctable.encode(str(sentence), output_len)
    
x_val = np.zeros((len(raw_prices_val), maxlen, len(chars)), dtype=np.bool)
y_val = np.zeros((len(parced_prices_val), output_len, len(chars)), dtype=np.bool)
for i, sentence in enumerate(raw_prices_val):
    x_val[i] = ctable.encode(str(sentence), maxlen)
for i, sentence in enumerate(parced_prices_val):
    y_val[i] = ctable.encode(str(sentence), output_len)

model = Sequential()
model.add(LSTM(config.hidden_size, input_shape=(maxlen, len(chars))))
model.add(RepeatVector(output_len))

model.add(BatchNormalization())

model.add(LSTM(config.hidden_size, return_sequences=True))
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
             metrics=['accuracy'])
model.summary()

# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=config.batch_size,
              epochs=1,
              validation_data=(x_val, y_val),callbacks=[WandbCallback()])
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print('☑', end=' ')
        else:
            print('☒', end=' ')
        print('G',guess, end='\n')


Total train prices: 575000
Total validation prices: 62
Vectorization...
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 256)               332800    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 10, 256)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 256)           1024      
_________________________________________________________________
lstm_3 (LSTM)                (None, 10, 256)           525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 10, 68)            17476     
Total params: 876,612
Trainable params: 876,100
Non-trainable params: 512
_________________________________________________________________

---------------------------------------